In [1]:
import numpy as np
import cv2  # OpenCVライブラリ

import matplotlib.pyplot as plt 
%matplotlib inline

import torch

import pickle

import torch.utils.data as data
from itertools import product as product

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function

In [2]:
# import dataset
from utils.dataset import VOCDataset, DatasetTransform, make_datapath_list, Anno_xml2list, od_collate_fn

In [3]:
# set your VOCdevkit path!
vocpath = "../VOCdevkit/VOC2007"
DEVKIT_PATH = "../VOCdevkit/"
SET = "test"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(vocpath)

In [15]:
val_img_list[0:10]

['../VOCdevkit/VOC2007/JPEGImages/000001.jpg',
 '../VOCdevkit/VOC2007/JPEGImages/000002.jpg',
 '../VOCdevkit/VOC2007/JPEGImages/000003.jpg',
 '../VOCdevkit/VOC2007/JPEGImages/000004.jpg',
 '../VOCdevkit/VOC2007/JPEGImages/000006.jpg',
 '../VOCdevkit/VOC2007/JPEGImages/000008.jpg',
 '../VOCdevkit/VOC2007/JPEGImages/000010.jpg',
 '../VOCdevkit/VOC2007/JPEGImages/000011.jpg',
 '../VOCdevkit/VOC2007/JPEGImages/000013.jpg',
 '../VOCdevkit/VOC2007/JPEGImages/000014.jpg']

In [14]:
val_anno_list[0:10]

['../VOCdevkit/VOC2007/Annotations/000001.xml',
 '../VOCdevkit/VOC2007/Annotations/000002.xml',
 '../VOCdevkit/VOC2007/Annotations/000003.xml',
 '../VOCdevkit/VOC2007/Annotations/000004.xml',
 '../VOCdevkit/VOC2007/Annotations/000006.xml',
 '../VOCdevkit/VOC2007/Annotations/000008.xml',
 '../VOCdevkit/VOC2007/Annotations/000010.xml',
 '../VOCdevkit/VOC2007/Annotations/000011.xml',
 '../VOCdevkit/VOC2007/Annotations/000013.xml',
 '../VOCdevkit/VOC2007/Annotations/000014.xml']

In [6]:
image_index = []
for l in val_img_list:
    image_index.append(l[-10:-4])

image_index[0]

'000001'

In [7]:
class_names = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする

## DatasetTransformを適応
transform = DatasetTransform(input_size, color_mean)
transform_anno = Anno_xml2list(class_names)

In [8]:
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DatasetTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(class_names))

In [16]:
val_dataloader = data.DataLoader(
    val_dataset, batch_size=1, shuffle=False, collate_fn=od_collate_fn, num_workers=1)

# set up model

In [17]:
from utils.feature_piramid_network import FPNSSD as SSD

voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDネットワークモデル
net = SSD(phase="inference", cfg=ssd_cfg).eval()

# SSDの学習済みの重みを設定
checkpoint = torch.load("./weights/ssd_fpn_300_200.pth")
net.load_state_dict(checkpoint["state_dict"])
#optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']
print('loaded the trained weights')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using:", device)

net = net.to(device)

loaded the trained weights
using: cuda:0


In [18]:
all_imgs = []
classes = {}
bbox_threshold = 0.05

# define detections
all_boxes = [[[] for _ in range(len(val_img_list))]
               for _ in range(21)]
empty_array = np.transpose(np.array([[],[],[],[],[]]), (1,0))

In [19]:
from utils.ssd_predict_show import SSDPredictShow
ssd = SSDPredictShow(eval_categories=voc_classes, net=net, device=device)

cuda:0


# infer images

In [ ]:
all_boxes = ssd.ssd_inference(val_dataloader, all_boxes, data_confidence_level=bbox_threshold)

../VOCdevkit/VOC2007/JPEGImages/000001.jpg
../VOCdevkit/VOC2007/JPEGImages/000002.jpg
../VOCdevkit/VOC2007/JPEGImages/000003.jpg
../VOCdevkit/VOC2007/JPEGImages/000004.jpg
../VOCdevkit/VOC2007/JPEGImages/000006.jpg
../VOCdevkit/VOC2007/JPEGImages/000008.jpg
../VOCdevkit/VOC2007/JPEGImages/000010.jpg
(1, 21, 200, 5)
iter: 0
sort boxes. detection was 0.03607940673828125 and post took 0.00015115737915039062 and allboxappend took 0.006963253021240234
(1, 21, 200, 5)
iter: 1
sort boxes. detection was 0.01636981964111328 and post took 7.152557373046875e-05 and allboxappend took 0.0064983367919921875
(1, 21, 200, 5)
iter: 2
sort boxes. detection was 0.05432271957397461 and post took 9.775161743164062e-05 and allboxappend took 0.006556987762451172
(1, 21, 200, 5)
iter: 3
sort boxes. detection was 0.05200767517089844 and post took 0.00010967254638671875 and allboxappend took 0.006685018539428711
../VOCdevkit/VOC2007/JPEGImages/000011.jpg
../VOCdevkit/VOC2007/JPEGImages/000013.jpg
../VOCdevkit/V

../VOCdevkit/VOC2007/JPEGImages/000090.jpg
../VOCdevkit/VOC2007/JPEGImages/000092.jpg
../VOCdevkit/VOC2007/JPEGImages/000094.jpg
../VOCdevkit/VOC2007/JPEGImages/000096.jpg
../VOCdevkit/VOC2007/JPEGImages/000097.jpg
../VOCdevkit/VOC2007/JPEGImages/000098.jpg
../VOCdevkit/VOC2007/JPEGImages/000100.jpg
(1, 21, 200, 5)
iter: 45
sort boxes. detection was 0.06113123893737793 and post took 0.00010275840759277344 and allboxappend took 0.006566762924194336
(1, 21, 200, 5)
iter: 46
sort boxes. detection was 0.03661465644836426 and post took 6.580352783203125e-05 and allboxappend took 0.006451845169067383
(1, 21, 200, 5)
iter: 47
sort boxes. detection was 0.017416954040527344 and post took 6.461143493652344e-05 and allboxappend took 0.006446123123168945
(1, 21, 200, 5)
iter: 48
sort boxes. detection was 0.025933504104614258 and post took 7.534027099609375e-05 and allboxappend took 0.0064563751220703125
(1, 21, 200, 5)
iter: 49
sort boxes. detection was 0.011776447296142578 and post took 6.1511993

../VOCdevkit/VOC2007/JPEGImages/000185.jpg
../VOCdevkit/VOC2007/JPEGImages/000186.jpg
../VOCdevkit/VOC2007/JPEGImages/000188.jpg
(1, 21, 200, 5)
iter: 87
sort boxes. detection was 0.07254552841186523 and post took 0.00011110305786132812 and allboxappend took 0.0065724849700927734
(1, 21, 200, 5)
iter: 88
sort boxes. detection was 0.06258940696716309 and post took 9.894371032714844e-05 and allboxappend took 0.006547212600708008
(1, 21, 200, 5)
iter: 89
sort boxes. detection was 0.06661701202392578 and post took 9.322166442871094e-05 and allboxappend took 0.006552457809448242
(1, 21, 200, 5)
iter: 90
sort boxes. detection was 0.016502857208251953 and post took 7.033348083496094e-05 and allboxappend took 0.006425380706787109
(1, 21, 200, 5)
iter: 91
sort boxes. detection was 0.010947465896606445 and post took 6.628036499023438e-05 and allboxappend took 0.0064697265625
../VOCdevkit/VOC2007/JPEGImages/000191.jpg
../VOCdevkit/VOC2007/JPEGImages/000195.jpg
../VOCdevkit/VOC2007/JPEGImages/0001

../VOCdevkit/VOC2007/JPEGImages/000277.jpg
../VOCdevkit/VOC2007/JPEGImages/000279.jpg
../VOCdevkit/VOC2007/JPEGImages/000280.jpg
../VOCdevkit/VOC2007/JPEGImages/000281.jpg
../VOCdevkit/VOC2007/JPEGImages/000283.jpg
iter: 133
sort boxes. detection was 0.018732786178588867 and post took 5.8650970458984375e-05 and allboxappend took 0.006651163101196289
(1, 21, 200, 5)
iter: 134
sort boxes. detection was 0.022764921188354492 and post took 5.9604644775390625e-05 and allboxappend took 0.006448984146118164
(1, 21, 200, 5)
iter: 135
sort boxes. detection was 0.011121749877929688 and post took 6.031990051269531e-05 and allboxappend took 0.006485939025878906
(1, 21, 200, 5)
iter: 136
sort boxes. detection was 0.04390215873718262 and post took 9.608268737792969e-05 and allboxappend took 0.0064547061920166016
(1, 21, 200, 5)
iter: 137
sort boxes. detection was 0.023571491241455078 and post took 7.82012939453125e-05 and allboxappend took 0.006509542465209961
../VOCdevkit/VOC2007/JPEGImages/000284.j

../VOCdevkit/VOC2007/JPEGImages/000365.jpg
../VOCdevkit/VOC2007/JPEGImages/000366.jpg
../VOCdevkit/VOC2007/JPEGImages/000368.jpg
../VOCdevkit/VOC2007/JPEGImages/000369.jpg
../VOCdevkit/VOC2007/JPEGImages/000371.jpg
../VOCdevkit/VOC2007/JPEGImages/000375.jpg
(1, 21, 200, 5)
iter: 179
sort boxes. detection was 0.03782010078430176 and post took 0.00011396408081054688 and allboxappend took 0.006485462188720703
(1, 21, 200, 5)
iter: 180
sort boxes. detection was 0.011396408081054688 and post took 9.036064147949219e-05 and allboxappend took 0.006530284881591797
(1, 21, 200, 5)
iter: 181
sort boxes. detection was 0.02889108657836914 and post took 0.00010848045349121094 and allboxappend took 0.006547689437866211
(1, 21, 200, 5)
iter: 182
sort boxes. detection was 0.012468814849853516 and post took 9.393692016601562e-05 and allboxappend took 0.006578207015991211
(1, 21, 200, 5)
iter: 183
sort boxes. detection was 0.024688005447387695 and post took 9.417533874511719e-05 and allboxappend took 0.0

...

../VOCdevkit/VOC2007/JPEGImages/001946.jpg
(1, 21, 200, 5)
iter: 972
sort boxes. detection was 0.07898807525634766 and post took 0.00010991096496582031 and allboxappend took 0.006757259368896484
(1, 21, 200, 5)
iter: 973
sort boxes. detection was 0.019280672073364258 and post took 0.00011277198791503906 and allboxappend took 0.0065822601318359375
(1, 21, 200, 5)
iter: 974
sort boxes. detection was 0.03968071937561035 and post took 8.630752563476562e-05 and allboxappend took 0.0064470767974853516
(1, 21, 200, 5)
iter: 975
sort boxes. detection was 0.015851497650146484 and post took 6.103515625e-05 and allboxappend took 0.006366729736328125
(1, 21, 200, 5)
iter: 976
sort boxes. detection was 0.012960433959960938 and post took 7.939338684082031e-05 and allboxappend took 0.006313323974609375
(1, 21, 200, 5)
iter: 977
sort boxes. detection was 0.03258681297302246 and post took 0.00010633468627929688 and allboxappend took 0.00665593147277832
(1, 21, 200, 5)
iter: 978
sort boxes. detection wa

../VOCdevkit/VOC2007/JPEGImages/002026.jpg
../VOCdevkit/VOC2007/JPEGImages/002028.jpg
../VOCdevkit/VOC2007/JPEGImages/002029.jpg
../VOCdevkit/VOC2007/JPEGImages/002031.jpg
../VOCdevkit/VOC2007/JPEGImages/002032.jpg
(1, 21, 200, 5)
iter: 1021
sort boxes. detection was 0.020052194595336914 and post took 7.343292236328125e-05 and allboxappend took 0.00651097297668457
(1, 21, 200, 5)
iter: 1022
sort boxes. detection was 0.010956287384033203 and post took 6.389617919921875e-05 and allboxappend took 0.00633549690246582
(1, 21, 200, 5)
iter: 1023
sort boxes. detection was 0.03186202049255371 and post took 8.177757263183594e-05 and allboxappend took 0.0064847469329833984
(1, 21, 200, 5)
iter: 1024
sort boxes. detection was 0.04960751533508301 and post took 9.489059448242188e-05 and allboxappend took 0.00659632682800293
(1, 21, 200, 5)
iter: 1025
sort boxes. detection was 0.04669070243835449 and post took 0.0003452301025390625 and allboxappend took 0.006383419036865234
(1, 21, 200, 5)
iter: 102

../VOCdevkit/VOC2007/JPEGImages/002106.jpg
../VOCdevkit/VOC2007/JPEGImages/002107.jpg
../VOCdevkit/VOC2007/JPEGImages/002110.jpg
../VOCdevkit/VOC2007/JPEGImages/002111.jpg
../VOCdevkit/VOC2007/JPEGImages/002113.jpg
../VOCdevkit/VOC2007/JPEGImages/002115.jpg
iter: 1064
sort boxes. detection was 0.06636476516723633 and post took 0.00010061264038085938 and allboxappend took 0.006907463073730469
(1, 21, 200, 5)
iter: 1065
sort boxes. detection was 0.012438774108886719 and post took 9.894371032714844e-05 and allboxappend took 0.006489992141723633
(1, 21, 200, 5)
iter: 1066
sort boxes. detection was 0.014299392700195312 and post took 9.202957153320312e-05 and allboxappend took 0.006594181060791016
(1, 21, 200, 5)
iter: 1067
sort boxes. detection was 0.011031627655029297 and post took 6.818771362304688e-05 and allboxappend took 0.006444692611694336
(1, 21, 200, 5)
iter: 1068
sort boxes. detection was 0.0695803165435791 and post took 0.0003483295440673828 and allboxappend took 0.00629878044128

../VOCdevkit/VOC2007/JPEGImages/002206.jpg
../VOCdevkit/VOC2007/JPEGImages/002207.jpg
(1, 21, 200, 5)
iter: 1105
sort boxes. detection was 0.02387213706970215 and post took 8.082389831542969e-05 and allboxappend took 0.006518125534057617
(1, 21, 200, 5)
iter: 1106
sort boxes. detection was 0.0471959114074707 and post took 0.00010824203491210938 and allboxappend took 0.0065765380859375
(1, 21, 200, 5)
iter: 1107
sort boxes. detection was 0.025117158889770508 and post took 0.00010752677917480469 and allboxappend took 0.00670623779296875
(1, 21, 200, 5)
iter: 1108
sort boxes. detection was 0.01885509490966797 and post took 8.153915405273438e-05 and allboxappend took 0.0065882205963134766
(1, 21, 200, 5)
iter: 1109
sort boxes. detection was 0.01118612289428711 and post took 7.033348083496094e-05 and allboxappend took 0.0063304901123046875
(1, 21, 200, 5)
iter: 1110
sort boxes. detection was 0.016000747680664062 and post took 8.58306884765625e-05 and allboxappend took 0.0064852237701416016


../VOCdevkit/VOC2007/JPEGImages/002298.jpg
../VOCdevkit/VOC2007/JPEGImages/002299.jpg
../VOCdevkit/VOC2007/JPEGImages/002301.jpg
../VOCdevkit/VOC2007/JPEGImages/002303.jpg
../VOCdevkit/VOC2007/JPEGImages/002304.jpg
(1, 21, 200, 5)
iter: 1151
sort boxes. detection was 0.015606403350830078 and post took 7.295608520507812e-05 and allboxappend took 0.006472349166870117
(1, 21, 200, 5)
iter: 1152
sort boxes. detection was 0.054108381271362305 and post took 9.655952453613281e-05 and allboxappend took 0.0063474178314208984
(1, 21, 200, 5)
iter: 1153
sort boxes. detection was 0.03822135925292969 and post took 8.869171142578125e-05 and allboxappend took 0.006555080413818359
(1, 21, 200, 5)
iter: 1154
sort boxes. detection was 0.025252342224121094 and post took 7.772445678710938e-05 and allboxappend took 0.006422996520996094
(1, 21, 200, 5)
iter: 1155
sort boxes. detection was 0.01235342025756836 and post took 6.103515625e-05 and allboxappend took 0.0065686702728271484
../VOCdevkit/VOC2007/JPEGI

In [ ]:
all_boxes[7][0:10]

# eval accuracy

In [ ]:
# eval function
def voc_eval(detpath,
             annopath,
             imagesetfile,
             classname,
             cachedir,
             ovthresh=0.5,
             use_07_metric=False):
  """
  rec, prec, ap = voc_eval(detpath,
                              annopath,
                              imagesetfile,
                              classname,
                              [ovthresh],
                              [use_07_metric])
  Top level function that does the PASCAL VOC evaluation.
  detpath: Path to detections
      detpath.format(classname) should produce the detection results file.
  annopath: Path to annotations
      annopath.format(imagename) should be the xml annotations file.
  imagesetfile: Text file containing the list of images, one image per line.
  classname: Category name (duh)
  cachedir: Directory for caching the annotations
  [ovthresh]: Overlap threshold (default = 0.5)
  [use_07_metric]: Whether to use VOC07's 11 point AP computation
      (default False)
  """
  # assumes detections are in detpath.format(classname)
  # assumes annotations are in annopath.format(imagename)
  # assumes imagesetfile is a text file with each line an image name
  # cachedir caches the annotations in a pickle file

  # first load gt
  if not os.path.isdir(cachedir):
    os.mkdir(cachedir)
  cachefile = os.path.join(cachedir, '%s_annots.pkl' % imagesetfile)
  # read list of images
  with open(imagesetfile, 'r') as f:
    lines = f.readlines()
  imagenames = [x.strip() for x in lines]

  if not os.path.isfile(cachefile):
    # load annotations
    recs = {}
    for i, imagename in enumerate(imagenames):
      recs[imagename] = parse_rec(annopath.format(imagename))
      if i % 100 == 0:
        print('Reading annotation for {:d}/{:d}'.format(
          i + 1, len(imagenames)))
    # save
    #print('Saving cached annotations to {:s}'.format(cachefile))
    #with open(cachefile, 'wb') as f:
    #  pickle.dump(recs, f)
  else:
    # load
    with open(cachefile, 'rb') as f:
      try:
        recs = pickle.load(f)
      except:
        recs = pickle.load(f, encoding='bytes')

  # extract gt objects for this class
  class_recs = {}
  npos = 0
  for imagename in imagenames:
    R = [obj for obj in recs[imagename] if obj['name'] == classname]
    bbox = np.array([x['bbox'] for x in R])
    difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
    det = [False] * len(R)
    npos = npos + sum(~difficult)
    class_recs[imagename] = {'bbox': bbox,
                             'difficult': difficult,
                             'det': det}

  # read dets
  detfile = detpath.format(classname)
  with open(detfile, 'r') as f:
    lines = f.readlines()

  splitlines = [x.strip().split(' ') for x in lines]
  image_ids = [x[0] for x in splitlines]
  confidence = np.array([float(x[1]) for x in splitlines])
  BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

  nd = len(image_ids)
  tp = np.zeros(nd)
  fp = np.zeros(nd)

  if BB.shape[0] > 0:
    # sort by confidence
    sorted_ind = np.argsort(-confidence)
#    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    for d in range(nd):
      id = image_ids[d][-10:-4]
      #print(id)
      # catch bad detections
      try:
          R = class_recs[id]
      except:
        print("det not found")
        continue
        
      bb = BB[d, :].astype(float)
      ovmax = -np.inf
      BBGT = R['bbox'].astype(float)

      if BBGT.size > 0:
        # compute overlaps
        # intersection
        ixmin = np.maximum(BBGT[:, 0], bb[0])
        iymin = np.maximum(BBGT[:, 1], bb[1])
        ixmax = np.minimum(BBGT[:, 2], bb[2])
        iymax = np.minimum(BBGT[:, 3], bb[3])
        iw = np.maximum(ixmax - ixmin + 1., 0.)
        ih = np.maximum(iymax - iymin + 1., 0.)
        inters = iw * ih

        # union
        uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
               (BBGT[:, 2] - BBGT[:, 0] + 1.) *
               (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

        overlaps = inters / uni
        ovmax = np.max(overlaps)
        jmax = np.argmax(overlaps)

      if ovmax > ovthresh:
        if not R['difficult'][jmax]:
          if not R['det'][jmax]:
            tp[d] = 1.
            R['det'][jmax] = 1
          else:
            fp[d] = 1.
      else:
        fp[d] = 1.

  # compute precision recall
  fp = np.cumsum(fp)
  tp = np.cumsum(tp)
  rec = tp / float(npos)
  # avoid divide by zero in case the first detection matches a difficult
  # ground truth
  prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
  ap = voc_ap(rec, prec, use_07_metric)

  return rec, prec, ap

In [ ]:
pascal_classes = np.asarray(["__background__", 'aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor'])

PASCAL_CLASSES = pascal_classes

# write out detections for evaluation

In [ ]:
import os 
def get_voc_results_file_template(cls):
        # VOCdevkit/results/VOC2007/Main/<comp_id>_det_test_aeroplane.txt
        filename = 'det_' + "val" + '_'+cls+'.txt'
        filedir = os.path.join(DEVKIT_PATH, 'results', 'VOC2007', 'Main')
        if not os.path.exists(filedir):
            os.makedirs(filedir)
        path = os.path.join(filedir, filename)
        return path


def write_voc_results_file(pascal_classes, all_boxes, image_index):
        for cls_ind, cls in enumerate(pascal_classes):
            if cls == '__background__':
                continue
            print('Writing {} VOC results file'.format(cls))
            filename = get_voc_results_file_template(cls)
            with open(filename, 'wt') as f:
                for im_ind, index in enumerate(image_index):
                    dets = np.asarray(all_boxes[cls_ind][im_ind])
                    if dets == []:
                        continue
                    # the VOCdevkit expects 1-based indices
                    for k in range(dets.shape[0]):
                        #print(dets[k, 0])
                        f.write('{:s} {:.3f} {:.1f} {:.1f} {:.1f} {:.1f}\n'.
                                format(index, dets[k, 0],
                                       dets[k, 1] + 1, dets[k, 2] + 1,
                                       dets[k, 3] + 1, dets[k, 4] + 1))
import xml.etree.ElementTree as ET
def parse_rec(filename):
  """ Parse a PASCAL VOC xml file """
  tree = ET.parse(filename)
  objects = []
  for obj in tree.findall('object'):
    obj_struct = {}
    obj_struct['name'] = obj.find('name').text
    obj_struct['pose'] = obj.find('pose').text
    obj_struct['truncated'] = int(obj.find('truncated').text)
    obj_struct['difficult'] = int(obj.find('difficult').text)
    bbox = obj.find('bndbox')
    obj_struct['bbox'] = [int(bbox.find('xmin').text),
                          int(bbox.find('ymin').text),
                          int(bbox.find('xmax').text),
                          int(bbox.find('ymax').text)]
    objects.append(obj_struct)

  return objects
def voc_ap(rec, prec, use_07_metric=False):
  """ ap = voc_ap(rec, prec, [use_07_metric])
  Compute VOC AP given precision and recall.
  If use_07_metric is true, uses the
  VOC 07 11 point method (default:False).
  """
  if use_07_metric:
    # 11 point metric
    ap = 0.
    for t in np.arange(0., 1.1, 0.1):
      if np.sum(rec >= t) == 0:
        p = 0
      else:
        p = np.max(prec[rec >= t])
      ap = ap + p / 11.
  else:
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], rec, [1.]))
    mpre = np.concatenate(([0.], prec, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
      mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
  return ap

In [ ]:
write_voc_results_file(pascal_classes, all_boxes, val_img_list)

# evaluation

In [ ]:
def python_eval(output_dir='output'):
        annopath = os.path.join(
            DEVKIT_PATH,
            'VOC2007',
            'Annotations',
            '{:s}.xml')
        imagesetfile = os.path.join(
            DEVKIT_PATH,
            'VOC2007',
            'ImageSets',
            'Main',
            SET + '.txt')
        cachedir = os.path.join(DEVKIT_PATH, 'annotations_cache')
        aps = []
        # The PASCAL VOC metric changed in 2010.
        # VOC07 metric is quite old so don't use.
        use_07_metric = False
        print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)
        for i, cls in enumerate(PASCAL_CLASSES):
            if cls == '__background__':
                continue
            filename = get_voc_results_file_template(cls)
            rec, prec, ap = voc_eval(
                filename, annopath, imagesetfile, cls, cachedir, ovthresh=0.5,
                use_07_metric=use_07_metric)
            aps += [ap]
            print('AP for {} = {:.4f}'.format(cls, ap))
            with open(os.path.join(output_dir, cls + '_pr.pkl'), 'wb') as f:
                pickle.dump({'rec': rec, 'prec': prec, 'ap': ap}, f)
        print('Mean AP = {:.4f}'.format(np.mean(aps)))
        print('~~~~~~~~')
        print('Results:')
        for ap in aps:
            print('{:.3f}'.format(ap))
        print('{:.3f}'.format(np.mean(aps)))
        print('~~~~~~~~')
        print('')
        print('--------------------------------------------------------------')
        print('Results computed with the **unofficial** Python eval code.')
        print('Results should be very close to the official MATLAB eval code.')
        print('Recompute with `./tools/reval.py --matlab ...` for your paper.')
        print('-- Thanks, The Management')
        print('--------------------------------------------------------------')

In [ ]:
# evaluate detections
python_eval()